In [14]:
# ============================================================================================
# PROBLEM C5
#
# Build and train a neural network model using the Daily Min Temperature.csv dataset.
# Use MAE as the metrics of your neural network model.
# We provided code for normalizing the data. Please do not change the code.
# Do not use lambda layers in your model.
#
# The dataset used in this problem is downloaded from https://github.com/jbrownlee/Datasets
#
# Desired MAE < 0.19 on the normalized dataset.
# ============================================================================================

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import csv
import urllib
import pandas as pd
from tensorflow.keras.optimizers import RMSprop

def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)


def solution_C5():
    df = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv')
    series = df["Temp"]

    # Normalization Function. DO NOT CHANGE THIS CODE
    min=np.min(series)
    max=np.max(series)
    series -= min
    series /= max

    # DO NOT CHANGE THIS CODE
    split_time=2500


    x_train=  series[:split_time]
    x_valid=  series[split_time:]
    # DO NOT CHANGE THIS CODE
    window_size=64
    batch_size=256
    shuffle_buffer_size=1000

    train_set=windowed_dataset(
        x_train, window_size, batch_size, shuffle_buffer_size)
    print(train_set)
    print(x_train.shape)

    valid_set=windowed_dataset(
        x_valid, window_size, batch_size, shuffle_buffer_size)
    print(train_set)
    print(x_train.shape)

    model=tf.keras.models.Sequential([
        tf.keras.layers.Conv1D(16,3,
                               strides = 1 ,
                               padding = "causal",
                               activation = "relu",
                               input_shape = [None,1]    ),

        tf.keras.layers.LSTM(64,return_sequences = True),
        tf.keras.layers.LSTM(64,return_sequences = True),
        tf.keras.layers.Dense(512, activation = 'relu'),
        tf.keras.layers.Dense(1),
    ])

    model.compile(loss = tf.keras.losses.Huber() ,
                  optimizer = RMSprop(learning_rate = 0.001), 
                  metrics = ["mae"])
    
    model.fit(train_set,
              epochs = 20, 
              validation_data =valid_set)
    # YOUR CODE HERE
    return model



# The code below is to save your model as a .h5 file.
# It will be saved automatically in your Submission folder.
if __name__ == '__main__':
    # DO NOT CHANGE THIS CODE
    model=solution_C5()
    model.save("model_C5.h5")


<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None))>
(2500,)
<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None))>
(2500,)
Epoch 1/20
10/10 [==============================] - 10s 587ms/step - loss: 0.0272 - mae: 0.1751 - val_loss: 0.0094 - val_mae: 0.1022
Epoch 2/20
10/10 [==============================] - 5s 477ms/step - loss: 0.0102 - mae: 0.1106 - val_loss: 0.0102 - val_mae: 0.1099
Epoch 3/20
10/10 [==============================] - 5s 482ms/step - loss: 0.0093 - mae: 0.1053 - val_loss: 0.0094 - val_mae: 0.1064
Epoch 4/20
10/10 [==============================] - 5s 478ms/step - loss: 0.0089 - mae: 0.1043 - val_loss: 0.0090 - val_mae: 0.1042
Epoch 5/20
10/10 [==============================] - 5s 479ms/step - loss: 0.0084 - mae: 0.1015 - val_loss: 0.0089 - val_mae: 0.1049
Epo